<style>
body {
    font-size: 20pt !important;
}

.rendered_html {
    font-size: 20pt !important;
}

.CodeMirror pre {
    font-size: 20pt !important;
}

.output pre {
    font-size: 20pt !important;
}
</style>


<h2 style="font-size:24pt"> Proyecto DESI</h2>

<h2 style="font-size:24pt"> Julio 3, 2025</h2>

<p style="font-size:16pt">
Conversión de datos desde RAC, DEC y z a coordenadas esféricas y finalmente a coordenadas x,y y z.
</p>


## Data

In [104]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.cosmology import Planck18
from astropy.io import ascii
import random

In [5]:
# original files are found here https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/
data_n = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_clustering.dat.fits")
data_s = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_clustering.dat.fits")

In [100]:
list_rand = list(range(18))
rand_tables_n = {}

for i in list_rand:
    url = f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_{i}_clustering.ran.fits"
    rand_tables_n[f"rand_{i}_n"] = Table.read(url)

In [101]:
rand_tables_s = {}

for i in list_rand:
    url = f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_{i}_clustering.ran.fits"
    rand_tables_s[f"rand_{i}_n"] = Table.read(url)

<p style="font-size:16pt">
$$\theta = 90 - \delta$$
$$\varphi = -\alpha$$
$$ z = r [Mpc] $$

In [20]:
def esfericas(data):
    data['theta']= 90 - data['DEC']
    data['phi'] = - data['RA']
    data['r'] = Planck18.comoving_distance(data['Z'])

In [21]:
esfericas(data_n)
esfericas(data_s)

<p style="font-size:16pt">
\begin{align*}
x &= r \sin\theta \cos\varphi \\
y &= r \sin\theta \sin\varphi \\
z &= r \cos\theta
\end{align*}


In [22]:
def cartesianas(data):
    data['x'] = data['r']*np.sin(data['theta'])*np.cos(data['phi'])
    data['y'] = data['r']*np.sin(data['theta'])*np.sin(data['phi'])
    data['z'] = data['r']*np.cos(data['theta'])

In [23]:
cartesianas(data_n)
cartesianas(data_s)

In [125]:
def save_file(number_rosette,data):
    
    #Choose random table
    clave_aleatoria = random.choice(list(rand_tables_n.keys()))
    rand_table = rand_tables_n[clave_aleatoria]

    #Filter of the rosette
    rosette = data[data['ROSETTE_NUMBER']==number_rosette]

    #Choose random index of the random table according to the len of rosette table
    len_rosette = len(rosette)
    idx_random = np.random.choice(len(rand_table), size=len_rosette, replace=False)
    rand_subset = rand_table[idx_random]

    #Add columns of the othe coordinates
    esfericas(rand_subset)
    cartesianas(rand_subset)

    #Filenames
    filename_data = f'data_rosette/LRG_{number_rosette}_clustering_data.ecsv'
    filename_rand = f'data_rosette/LRG_{number_rosette}_clustering_rand.ecsv'

    #Write
    ascii.write(rosette, filename_data, format='ecsv', overwrite=True)
    ascii.write(rand_subset, filename_rand, format='ecsv', overwrite=True)

In [126]:
#Save files north
for i in set(data_n['ROSETTE_NUMBER']):
    save_file(i,data_n)

In [127]:
#Save files south
for i in set(data_s['ROSETTE_NUMBER']):
    save_file(i,data_s)